# FESTA Demo - LLaVA 1.6 7B with Unsloth (2x Faster!)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iiscleap/mllm-uncertainty-estimation/blob/main/festa_demo/FESTA_Unsloth_Demo.ipynb)

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
</div>

**Optimized FESTA notebook using Unsloth for 2-5x faster inference with 70% less memory!**

Test LLaVA 1.6 7B model with FESTA example images using research-grade prompting.

In [ ]:
%%capture
import os
# Install Unsloth for optimized inference
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Optimized Colab installation
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install pillow requests matplotlib

In [ ]:
from unsloth import FastLanguageModel
import torch
import json
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
import warnings
from transformers import logging

# Suppress warnings (same as research script)
warnings.filterwarnings("ignore")
logging.set_verbosity_error()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Unsloth optimized model loading
print("Loading Unsloth optimized LLaVA model...")

max_seq_length = 2048  # Choose any! Auto supports RoPE Scaling
dtype = None           # None for auto detection. Float16 for Tesla T4
load_in_4bit = True    # Use 4bit quantization for memory efficiency

# Load the Unsloth optimized LLaVA model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llava-v1.6-mistral-7b-hf",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

print("✅ Unsloth optimized LLaVA model loaded successfully!")
print("🚀 2-5x faster inference with 70% less memory usage!")

In [ ]:
# GitHub base URL for examples
base_url = "https://raw.githubusercontent.com/iiscleap/mllm-uncertainty-estimation/main/festa_demo/examples/"

def load_image_from_url(image_name):
    """Load image from GitHub repository"""
    url = base_url + image_name
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def create_llava_prompt(question, dataset_type="blink"):
    """Create LLaVA prompt with exact same format as research script"""
    
    # Set choices based on dataset (same as research script)
    if dataset_type == "blink":
        choices = "A. Yes\nB. No"
    elif dataset_type == "vsr":
        choices = "A. True\nB. False"
    else:
        choices = "A. Yes\nB. No"
    
    # Create instruction with exact same format as research script
    instruction = f"{question}\nChoices:\n{choices}\nReturn only the option (A or B), and nothing else.\nMAKE SURE your output is A or B"
    
    # LLaVA conversation format
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image"},
            ],
        },
    ]
    
    return conversation

def generate_response_unsloth(image, question, dataset_type="blink"):
    """Generate response using Unsloth optimized LLaVA with research-grade prompting"""
    
    conversation = create_llava_prompt(question, dataset_type)
    
    # Apply chat template (same as research script)
    prompt = tokenizer.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = tokenizer(images=image, text=prompt, return_tensors="pt").to(device)
    
    # Generate with Unsloth optimizations (same parameters as research script)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1,  # Single token A/B response
            use_cache=True,    # Unsloth optimization
            do_sample=False,   # Deterministic
            temperature=1.0,
        )
    
    # Extract response (same as research script)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()[-1]
    
    return response

def test_example_unsloth(image_name, question, title, dataset_type="blink"):
    """Test example with Unsloth optimized inference"""
    image = load_image_from_url(image_name)
    
    # Measure inference time
    import time
    start_time = time.time()
    response = generate_response_unsloth(image, question, dataset_type)
    inference_time = time.time() - start_time
    
    # Determine full answer text
    if dataset_type == "blink":
        full_answer = "A (Yes)" if response == "A" else "B (No)" if response == "B" else response
    elif dataset_type == "vsr":
        full_answer = "A (True)" if response == "A" else "B (False)" if response == "B" else response
    else:
        full_answer = "A (Yes)" if response == "A" else "B (No)" if response == "B" else response
    
    # Display results
    plt.figure(figsize=(12, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{title}\nQuestion: {question}\nLLaVA Answer: {full_answer}\n⚡ Inference Time: {inference_time:.2f}s", 
              fontsize=12, pad=20)
    plt.tight_layout()
    plt.show()
    
    print(f"🚀 Unsloth optimized inference: {inference_time:.2f}s")
    
    return response

print("✅ Unsloth optimized functions loaded with research-grade prompting!")

## 🚀 Test with 6 FESTA Examples (Unsloth Optimized)

Using **Unsloth optimizations** with the exact same prompting setup as the FESTA research paper.

In [ ]:
# Example 1: Original Spatial Relation
print("🔍 Testing Example 1 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_1.jpg",
    "Is the car beneath the cat?",
    "Example 1: Original Spatial Relation (Unsloth Optimized)",
    "blink"
)
print(f"Raw Response: {response}\n")

In [ ]:
# Example 2: Contrast Perturbation (Equivalent Sample)
print("🔍 Testing Example 2 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_1_contrast1.jpg",
    "Is the car beneath the cat?",
    "Example 2: Contrast Perturbation (Should be same as Example 1)",
    "blink"
)
print(f"Raw Response: {response}\n")

In [ ]:
# Example 3: Masking Perturbation (Equivalent Sample)
print("🔍 Testing Example 3 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_1_masking1.jpg",
    "Is the car beneath the cat?",
    "Example 3: Masking Perturbation (Should be same as Example 1)",
    "blink"
)
print(f"Raw Response: {response}\n")

In [ ]:
# Example 4: Negated/Complementary Version (Should toggle answer)
print("🔍 Testing Example 4 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_1_negated_contrast1.jpg",
    "Is the car beneath the cat?",
    "Example 4: Negated Scene (Should give opposite answer)",
    "blink"
)
print(f"Raw Response: {response}\n")

In [ ]:
# Example 5: Different Scene Original
print("🔍 Testing Example 5 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_5.jpg",
    "Are there animals in this image?",
    "Example 5: Different Scene Original",
    "blink"
)
print(f"Raw Response: {response}\n")

In [ ]:
# Example 6: Different Scene Blur (Equivalent Sample)
print("🔍 Testing Example 6 with Unsloth optimizations...")
response = test_example_unsloth(
    "val_Spatial_Relation_5_blur1.jpg",
    "Are there animals in this image?",
    "Example 6: Blur Perturbation (Should be same as Example 5)",
    "blink"
)
print(f"Raw Response: {response}\n")

## ⚡ FESTA Consistency Test (Unsloth Optimized)

Test equivalent sampling with optimized performance:

In [ ]:
# FESTA Equivalent Sampling Test with performance tracking
print("🔍 FESTA Equivalent Sampling Test (Unsloth Optimized):")
print("Testing same question on original vs perturbed images (should be consistent)\n")

import time
question = "Is the car beneath the cat?"
images = [
    "val_Spatial_Relation_1.jpg",
    "val_Spatial_Relation_1_contrast1.jpg", 
    "val_Spatial_Relation_1_masking1.jpg"
]

responses = []
total_time = 0

for i, img in enumerate(images):
    start_time = time.time()
    image = load_image_from_url(img)
    resp = generate_response_unsloth(image, question, "blink")
    inference_time = time.time() - start_time
    
    responses.append(resp)
    total_time += inference_time
    
    img_type = ["Original", "Contrast", "Masking"][i]
    print(f"{img_type:>10}: {resp} (⚡ {inference_time:.2f}s)")

# Check consistency
all_same = len(set(responses)) == 1
print(f"\n{'✅ CONSISTENT' if all_same else '❌ INCONSISTENT'}: {'All responses match' if all_same else 'Responses vary across equivalent samples'}")
print(f"🚀 Total inference time with Unsloth: {total_time:.2f}s")
print(f"⚡ Average time per sample: {total_time/3:.2f}s")

if not all_same:
    print("⚠️  FESTA Equivalent Sampling Failure Detected!")
else:
    print("🎉 Model shows consistency across equivalent perturbations!")

In [ ]:
# Benchmark: Compare speeds (Optional)
def benchmark_inference():
    """Quick benchmark of Unsloth optimized inference"""
    print("🏁 Benchmarking Unsloth optimized inference...")
    
    image = load_image_from_url("val_Spatial_Relation_1.jpg")
    question = "Is the car beneath the cat?"
    
    # Run 5 inference samples
    times = []
    for i in range(5):
        start_time = time.time()
        response = generate_response_unsloth(image, question, "blink")
        inference_time = time.time() - start_time
        times.append(inference_time)
        print(f"Run {i+1}: {response} (⚡ {inference_time:.2f}s)")
    
    avg_time = sum(times) / len(times)
    print(f"\n🚀 Average Unsloth inference time: {avg_time:.2f}s")
    print(f"⚡ 2-5x faster than standard transformers!")

# Uncomment to run benchmark
# benchmark_inference()

In [ ]:
# Custom testing function with Unsloth optimizations
def test_custom_unsloth():
    """Custom testing with Unsloth optimized inference"""
    # Change these values to test other images/questions
    image_name = "val_Spatial_Relation_1.jpg"  # Change this
    question = "Is the car beneath the cat?"      # Change this
    dataset_type = "blink"                       # "blink" or "vsr"
    
    print("🔍 Custom test with Unsloth optimizations...")
    response = test_example_unsloth(image_name, question, "Custom Test (Unsloth)", dataset_type)
    print(f"Custom Response: {response}")

# Uncomment to use custom testing
# test_custom_unsloth()

## 📊 Performance & Research Notes

### 🚀 **Unsloth Optimizations:**
- **2-5x faster inference** compared to standard transformers
- **70% less memory usage** with 4-bit quantization
- **Native optimizations** for Google Colab T4 GPUs
- **Same model quality** as original LLaVA 1.6 7B

### 🔬 **Research Fidelity:**
This notebook uses the **exact same prompting setup** as the FESTA research paper:

- **System Prompt**: `"{question}\nChoices:\n{choices}\nReturn only the option (A or B), and nothing else.\nMAKE SURE your output is A or B"`
- **Chat Template**: Applied via `tokenizer.apply_chat_template()`
- **Generation**: `max_new_tokens=1` for single token A/B response
- **Model**: `unsloth/llava-v1.6-mistral-7b-hf` with optimizations

### 🎯 **FESTA Framework Tests:**
- **Equivalent Samples**: Same question, different perturbations → Should give consistent answers
- **Complementary Samples**: Opposite scenarios → Should give different answers

### 🔗 **Links:**
- **Unsloth**: https://unsloth.ai/
- **Model**: https://huggingface.co/unsloth/llava-v1.6-mistral-7b-hf
- **Discord**: https://discord.gg/unsloth

---
*Optimized with ❤️ by Unsloth AI for 2-5x faster inference!*